## Collection of simple python snippets used for Telecom related dataset analysis


### Snippet Number 02 - Start

* Cleaning up the Nulls

For most of the Telecom related datasets (excel files, csv files, texts, exports etc.) there are a lot of different notations used by systems to indicate NULLs.

**Huawei** may use the text **NIL**, Other vendors may use **NULL** or **/0**
Then there are other artifacts due to Excel formulas such as **#VALUE!** errors etc.

The problem is that for analysis using Pandas, if we have such values, then Pandas will not consider this column as a Numeric column. Cell/Site/MSC/MME level data is actually composed of mostly Numeric values only with exception of the MME/MSC/RNC names or Site names which are categorical.

So we should try to remove all Knowns NULL values from the dataset and try to convert those columns to Numeric for a better data handling.

In [1]:
#################################################################################
# If we find any below value in a row, we will convert it to np.nan 
# and then try to convert the whole column into a numeric pandas type.
#################################################################################
KNOWN_NA_VALS = ['NIL', 'NILL', 'NULL', 'NA', '#NA', '#N/A', 
                 'N/A','#VALUE!','#REF!','#DIV/0!','#NUM!',
                 '#NAME?','#NULL!','NAN','nan','NaN', '#REF', 
                 '#DIV/0','#VALUE','#REF','#NUM', '#NULL']
#################################################################################


import pandas as pd
import numpy as np

def cleanup_nulls(dataframe):
    """
    This function tries to substitute known NULL values and then tries to convert the columns into numeric types
    :param df: DataFrame as input
    :return: Cleaned dataframe as output
    """
    df = dataframe.copy() # Make a copy to eliminate reference based issues
    try:
        print("*" * 100)
        print("Current data frame has got {} non-numeric/datetime columns".format(
                    len(df.select_dtypes(exclude=["number", "datetime"]).columns)))
        print("*" * 100)
        for x in df.select_dtypes(exclude=["number", "datetime"]).columns:
            if len(df[df[x].isin(KNOWN_NA_VALS)]) > 0: #some instances were found with Known NA substitutions
                temp_col = df[x].copy() #Copy data else it would create references of df
                temp_col[ temp_col.isin(KNOWN_NA_VALS)] = np.nan
                try: #now we will try to see if the column can become numeric
                    temp_col = pd.to_numeric(temp_col, errors='raise')
                    df[x] = temp_col.copy()
                    print("-\tSuccess in converting {} into a numeric column".format(x))
                except ValueError as e:
                    print("-\tFailure in converting {} into a numeric column".format(x))
                    continue #Column cannot be converted to numeric. Just continue
        print("*" * 100)
        print("After processing the data frame has got {} non-numeric/datetime columns".format(
                    len(df.select_dtypes(exclude=["number", "datetime"]).columns)))
        print("*" * 100)
        return df
    except Exception as broad_exception:
        print("An exception occurred in the process of cleanup. {}".format(broad_exception))
        return None


#### Usage (Read a file then call cleanup_nulls function)

In [2]:
df = pd.read_csv("data/sample_input_file.csv", low_memory=False) #just skipping a row for fun (no headers now)

In [3]:
df = cleanup_nulls(df)

****************************************************************************************************
Current data frame has got 9 non-numeric/datetime columns
****************************************************************************************************
-	Success in converting DCR EUL into a numeric column
-	Success in converting FACH URA SR % into a numeric column
-	Success in converting R99 PS DCR into a numeric column
-	Success in converting Total HS Users (HS/R99 and HS/EUL) into a numeric column
-	Success in converting Avg. HS Users (HS/R99 and HS/EUL) into a numeric column
****************************************************************************************************
After processing the data frame has got 4 non-numeric/datetime columns
****************************************************************************************************


### Snippet Number 02 - End